# Step 1: Reference Data Analysis - 500×500m Districts

**Goal**: Load and analyze reference street networks from 4 cities with comprehensive visualizations

**Cities**:
- London, UK
- Berlin, Germany  
- Belgrade, Serbia
- Torino, Italy

**Visualizations**:
- Network plots (properly projected)
- Morphology metrics (filtered data)
- Polar orientation diagrams
- Space syntax maps (integration, choice, mean depth)
- Intelligibility scatter plots
- Cross-city comparisons

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pathlib import Path
from collections import Counter
import math
import pickle
from shapely.geometry import Point, LineString
from shapely.ops import transform
import pyproj

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

## Configuration

In [ ]:
CITIES = {
    'london': {'name': 'London, UK', 'color': '#E74C3C'},
    'berlin': {'name': 'Berlin, Germany', 'color': '#3498DB'},
    'belgrade': {'name': 'Belgrade, Serbia', 'color': '#2ECC71'},
    'torino': {'name': 'Torino, Italy', 'color': '#F39C12'}
}

WINDOW_SIZE_M = 500
DATA_DIR = Path("inv_city/outputs/geojson")
MIN_SEGMENT_LENGTH = 5.0  # Filter out segments < 5m

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print(f"Min segment length: {MIN_SEGMENT_LENGTH}m")
print(f"Data directory: {DATA_DIR}")

## Helper Functions

In [ ]:
def calculate_bearing(p1, p2):
    """Calculate bearing (0-180°) of line segment."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx)
    bearing = math.degrees(angle)
    if bearing < 0:
        bearing += 180
    if bearing >= 180:
        bearing -= 180
    return bearing


def reproject_to_utm(gdf, city_key):
    """Reproject GeoDataFrame to local UTM zone."""
    # Get center point
    centroid = gdf.geometry.unary_union.centroid
    
    # Determine UTM zone
    lon = centroid.x
    lat = centroid.y
    utm_zone = int((lon + 180) / 6) + 1
    hemisphere = 'north' if lat >= 0 else 'south'
    
    # Create UTM CRS
    utm_crs = f"+proj=utm +zone={utm_zone} +{hemisphere} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Reproject
    gdf_utm = gdf.to_crs(utm_crs)
    
    return gdf_utm, utm_crs


def normalize_to_origin(gdf):
    """Shift coordinates so bounding box starts at origin."""
    bounds = gdf.total_bounds  # minx, miny, maxx, maxy
    minx, miny = bounds[0], bounds[1]
    
    # Shift geometry
    gdf_shifted = gdf.copy()
    gdf_shifted.geometry = gdf_shifted.geometry.translate(xoff=-minx, yoff=-miny)
    
    return gdf_shifted


def load_and_clean_network(city_key):
    """Load network from GeoJSON, reproject, filter, and clean."""
    edges_file = DATA_DIR / f"{city_key}_edges.geojson"
    nodes_file = DATA_DIR / f"{city_key}_nodes.geojson"
    
    if not edges_file.exists() or not nodes_file.exists():
        raise FileNotFoundError(f"Missing data for {city_key}")
    
    # Load
    edges_gdf = gpd.read_file(edges_file)
    nodes_gdf = gpd.read_file(nodes_file)
    
    print(f"  Raw: {len(nodes_gdf)} nodes, {len(edges_gdf)} edges")
    
    # Reproject to UTM
    edges_utm, utm_crs = reproject_to_utm(edges_gdf, city_key)
    nodes_utm = nodes_gdf.to_crs(utm_crs)
    
    # Normalize to origin
    edges_utm = normalize_to_origin(edges_utm)
    nodes_utm = normalize_to_origin(nodes_utm)
    
    # Filter short segments
    edges_utm = edges_utm[edges_utm.geometry.length >= MIN_SEGMENT_LENGTH].copy()
    print(f"  After filtering (<{MIN_SEGMENT_LENGTH}m): {len(edges_utm)} edges")
    
    # Build graph from cleaned data
    G = nx.Graph()
    pos = {}
    
    # Add nodes
    for idx, row in nodes_utm.iterrows():
        geom = row.geometry
        pos[idx] = (geom.x, geom.y)
        G.add_node(idx, x=geom.x, y=geom.y)
    
    # Add edges
    for idx, row in edges_utm.iterrows():
        geom = row.geometry
        coords = list(geom.coords)
        start = coords[0]
        end = coords[-1]
        
        u = find_closest_node(start, pos)
        v = find_closest_node(end, pos)
        
        if u is not None and v is not None and u != v:
            length = geom.length
            G.add_edge(u, v, length=length)
    
    # Remove isolated nodes
    isolated = list(nx.isolates(G))
    G.remove_nodes_from(isolated)
    for node in isolated:
        if node in pos:
            del pos[node]
    
    print(f"  Final: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    return G, pos


def find_closest_node(point, pos, tolerance=5.0):
    """Find closest node to a point."""
    min_dist = float('inf')
    closest = None
    
    for node_id, node_pos in pos.items():
        dist = math.sqrt((point[0] - node_pos[0])**2 + (point[1] - node_pos[1])**2)
        if dist < min_dist:
            min_dist = dist
            closest = node_id
    
    return closest if min_dist <= tolerance else None


def compute_morphology_metrics(G, pos):
    """Compute all morphology metrics."""
    metrics = {}
    
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    metrics['node_density'] = G.number_of_nodes() / area_km2
    
    degrees = [d for _, d in G.degree()]
    metrics['degree_distribution'] = dict(Counter(degrees))
    metrics['avg_degree'] = np.mean(degrees) if degrees else 0
    
    dead_ends = sum(1 for d in degrees if d == 1)
    metrics['dead_end_ratio'] = dead_ends / len(degrees) if degrees else 0
    
    lengths = []
    for u, v in G.edges():
        length = np.linalg.norm(np.array(pos[u]) - np.array(pos[v]))
        lengths.append(length)
    
    metrics['segment_lengths'] = lengths
    metrics['avg_segment_length'] = np.mean(lengths) if lengths else 0
    
    bearings = []
    for u, v in G.edges():
        bearing = calculate_bearing(pos[u], pos[v])
        bearings.append(bearing)
    
    if bearings:
        counts, bins = np.histogram(bearings, bins=18, range=(0, 180))
        metrics['orientation_hist'] = (bins, counts)
        metrics['bearings'] = bearings
    else:
        metrics['orientation_hist'] = (np.linspace(0, 180, 19), np.zeros(18))
        metrics['bearings'] = []
    
    return metrics


def compute_space_syntax_metrics(G):
    """Compute space syntax metrics."""
    if G.number_of_nodes() < 2:
        return {'mean_depth': 0, 'local_integration': {}, 'choice': {}, 'intelligibility': 0}
    
    if not nx.is_connected(G):
        largest_cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(largest_cc).copy()
    
    # Mean depth
    total_depth = 0
    count = 0
    for source in G.nodes():
        lengths = nx.single_source_shortest_path_length(G, source)
        total_depth += sum(lengths.values())
        count += len(lengths)
    
    mean_depth = total_depth / count if count > 0 else 0
    
    # Mean depth per node (for visualization)
    mean_depth_per_node = {}
    for node in G.nodes():
        lengths = nx.single_source_shortest_path_length(G, node)
        avg = np.mean(list(lengths.values())) if lengths else 0
        mean_depth_per_node[node] = avg
    
    # Local integration (radius 3)
    local_int = {}
    for node in G.nodes():
        lengths = nx.single_source_shortest_path_length(G, node, cutoff=3)
        if len(lengths) > 1:
            total = sum(lengths.values())
            local_int[node] = (len(lengths) - 1) / total if total > 0 else 0
        else:
            local_int[node] = 0
    
    # Choice (betweenness)
    choice = nx.betweenness_centrality(G, normalized=True)
    
    # Intelligibility
    degrees = [G.degree(n) for n in local_int.keys()]
    integrations = list(local_int.values())
    
    if len(degrees) > 1 and np.std(degrees) > 0 and np.std(integrations) > 0:
        corr = np.corrcoef(degrees, integrations)[0, 1]
    else:
        corr = 0
    
    return {
        'mean_depth': mean_depth,
        'mean_depth_per_node': mean_depth_per_node,
        'local_integration': local_int,
        'choice': choice,
        'intelligibility': corr
    }


print("✓ Helper functions defined")

## Load All Reference Cities

In [ ]:
city_data = {}

print("Loading and cleaning reference cities...\n")
print("="*70)

for city_key in CITIES.keys():
    print(f"\n{CITIES[city_key]['name']}:")
    try:
        G, pos = load_and_clean_network(city_key)
        morph = compute_morphology_metrics(G, pos)
        syntax = compute_space_syntax_metrics(G)
        
        city_data[city_key] = {
            'graph': G,
            'pos': pos,
            'morphology': morph,
            'syntax': syntax
        }
        
    except Exception as e:
        print(f"  ✗ Error: {e}")

print("\n" + "="*70)
print(f"✓ Loaded {len(city_data)} cities successfully\n")

## A1. Visualize All 4 Networks (Fixed Coordinates)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    color = CITIES[city_key]['color']
    
    # Get actual bounds from positions
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    
    if all_x and all_y:
        minx, maxx = min(all_x), max(all_x)
        miny, maxy = min(all_y), max(all_y)
        
        # Draw window boundary
        ax.add_patch(Rectangle(
            (minx, miny), maxx-minx, maxy-miny,
            fill=False, edgecolor='gray', linestyle='--', linewidth=1.5
        ))
        
        # Draw edges
        for u, v in G.edges():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color=color, linewidth=1.5, alpha=0.7, zorder=1)
        
        # Draw nodes colored by degree
        degrees = dict(G.degree())
        max_degree = max(degrees.values()) if degrees else 1
        
        for node in G.nodes():
            degree = degrees[node]
            color_val = degree / max_degree
            node_color = plt.cm.RdYlBu_r(color_val)
            
            ax.scatter(
                pos[node][0], pos[node][1],
                s=40, c=[node_color], zorder=2,
                edgecolors='black', linewidths=0.5
            )
        
        ax.set_xlim(minx - 20, maxx + 20)
        ax.set_ylim(miny - 20, maxy + 20)
    
    ax.set_aspect('equal')
    ax.set_title(
        f"{CITIES[city_key]['name']}\n{G.number_of_nodes()} nodes, {G.number_of_edges()} edges",
        fontsize=12, fontweight='bold'
    )
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Reference Street Networks (nodes colored by degree)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A2. Summary Statistics

In [ ]:
summary_rows = []

for city_key, data in city_data.items():
    G = data['graph']
    morph = data['morphology']
    syntax = data['syntax']
    
    summary_rows.append({
        'City': CITIES[city_key]['name'],
        'Nodes': G.number_of_nodes(),
        'Edges': G.number_of_edges(),
        'Density\n(n/km²)': f"{morph['node_density']:.1f}",
        'Avg\nDegree': f"{morph['avg_degree']:.2f}",
        'Dead-End\nRatio': f"{morph['dead_end_ratio']:.3f}",
        'Avg Seg\nLength (m)': f"{morph['avg_segment_length']:.1f}",
        'Mean\nDepth': f"{syntax['mean_depth']:.2f}",
        'Intelligibility': f"{syntax['intelligibility']:.3f}"
    })

df_summary = pd.DataFrame(summary_rows)
print("\n" + "="*100)
print(" "*35 + "REFERENCE CITIES SUMMARY")
print("="*100)
print(df_summary.to_string(index=False))
print("="*100)

## A3. Degree Distribution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    degree_dist = data['morphology']['degree_distribution']
    
    degrees = sorted(degree_dist.keys())
    counts = [degree_dist[d] for d in degrees]
    total = sum(counts)
    probs = [c / total for c in counts]
    
    color = CITIES[city_key]['color']
    ax.bar(degrees, probs, color=color, alpha=0.7, edgecolor='black', linewidth=1)
    ax.set_xlabel('Node Degree', fontsize=11)
    ax.set_ylabel('Probability', fontsize=11)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks(degrees)

plt.suptitle('Degree Distributions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## A4. Orientation as Polar/Rose Diagrams

In [ ]:
fig = plt.figure(figsize=(14, 10))

for idx, (city_key, data) in enumerate(city_data.items(), 1):
    ax = fig.add_subplot(2, 2, idx, projection='polar')
    
    bins, counts = data['morphology']['orientation_hist']
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Convert to radians and double for full circle
    theta = np.deg2rad(bin_centers)
    theta_full = np.concatenate([theta, theta + np.pi])
    counts_full = np.concatenate([counts, counts])
    
    # Normalize
    total = sum(counts_full)
    probs = counts_full / total if total > 0 else counts_full
    
    color = CITIES[city_key]['color']
    width = np.deg2rad(bins[1] - bins[0])
    
    ax.bar(theta_full, probs, width=width, color=color, alpha=0.7, edgecolor='black', linewidth=0.5)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=12, pad=20)
    ax.set_ylim(0, max(probs) * 1.1 if max(probs) > 0 else 0.1)

plt.suptitle('Street Orientation Rose Diagrams', fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

## A5. Local Integration Maps (Color-Coded Networks)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    local_int = data['syntax']['local_integration']
    
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    
    if all_x and all_y:
        minx, maxx = min(all_x), max(all_x)
        miny, maxy = min(all_y), max(all_y)
        
        # Draw edges (gray)
        for u, v in G.edges():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.5, zorder=1)
        
        # Draw nodes colored by local integration
        values = list(local_int.values())
        if values:
            vmin, vmax = min(values), max(values)
            
            for node in G.nodes():
                if node in local_int:
                    val = local_int[node]
                    norm_val = (val - vmin) / (vmax - vmin + 1e-10)
                    color = plt.cm.hot(norm_val)
                    
                    ax.scatter(
                        pos[node][0], pos[node][1],
                        s=60, c=[color], zorder=2,
                        edgecolors='black', linewidths=0.5
                    )
        
        ax.set_xlim(minx - 20, maxx + 20)
        ax.set_ylim(miny - 20, maxy + 20)
    
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nLocal Integration (R=3)", fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Local Integration Maps (warm = high integration)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A6. Choice (Betweenness) Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    choice = data['syntax']['choice']
    
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    
    if all_x and all_y:
        minx, maxx = min(all_x), max(all_x)
        miny, maxy = min(all_y), max(all_y)
        
        # Draw edges (gray)
        for u, v in G.edges():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.5, zorder=1)
        
        # Draw nodes colored by choice
        values = list(choice.values())
        if values:
            vmin, vmax = min(values), max(values)
            
            for node in G.nodes():
                if node in choice:
                    val = choice[node]
                    norm_val = (val - vmin) / (vmax - vmin + 1e-10)
                    color = plt.cm.viridis(norm_val)
                    
                    ax.scatter(
                        pos[node][0], pos[node][1],
                        s=60, c=[color], zorder=2,
                        edgecolors='black', linewidths=0.5
                    )
        
        ax.set_xlim(minx - 20, maxx + 20)
        ax.set_ylim(miny - 20, maxy + 20)
    
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nChoice (Betweenness)", fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Choice Maps (through-movement corridors)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A7. Mean Depth Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    mean_depth_nodes = data['syntax']['mean_depth_per_node']
    
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    
    if all_x and all_y:
        minx, maxx = min(all_x), max(all_x)
        miny, maxy = min(all_y), max(all_y)
        
        # Draw edges (gray)
        for u, v in G.edges():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.5, zorder=1)
        
        # Draw nodes colored by mean depth
        values = list(mean_depth_nodes.values())
        if values:
            vmin, vmax = min(values), max(values)
            
            for node in G.nodes():
                if node in mean_depth_nodes:
                    val = mean_depth_nodes[node]
                    norm_val = (val - vmin) / (vmax - vmin + 1e-10)
                    color = plt.cm.coolwarm_r(norm_val)
                    
                    ax.scatter(
                        pos[node][0], pos[node][1],
                        s=60, c=[color], zorder=2,
                        edgecolors='black', linewidths=0.5
                    )
        
        ax.set_xlim(minx - 20, maxx + 20)
        ax.set_ylim(miny - 20, maxy + 20)
    
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nMean Depth", fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Mean Depth Maps (blue = shallow/central, red = deep/peripheral)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A8. Intelligibility Scatter Plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    local_int = data['syntax']['local_integration']
    intelligibility = data['syntax']['intelligibility']
    
    # Get degrees and integration values
    degrees = [G.degree(n) for n in local_int.keys()]
    integrations = list(local_int.values())
    
    color = CITIES[city_key]['color']
    ax.scatter(degrees, integrations, alpha=0.6, s=30, c=color, edgecolors='black', linewidths=0.5)
    
    # Fit line
    if len(degrees) > 1:
        z = np.polyfit(degrees, integrations, 1)
        p = np.poly1d(z)
        x_line = np.linspace(min(degrees), max(degrees), 100)
        ax.plot(x_line, p(x_line), 'r--', linewidth=2, alpha=0.7)
    
    ax.set_xlabel('Connectivity (Degree)', fontsize=11)
    ax.set_ylabel('Local Integration', fontsize=11)
    ax.set_title(
        f"{CITIES[city_key]['name']}\nIntelligibility r = {intelligibility:.3f}",
        fontweight='bold', fontsize=12
    )
    ax.grid(True, alpha=0.3)

plt.suptitle('Intelligibility: Degree vs Local Integration', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## A9. Cross-City Small Multiples (Key Comparison)

In [ ]:
# Segment length comparison
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    lengths = data['morphology']['segment_lengths']
    
    if lengths:
        counts, bins = np.histogram(lengths, bins=20, range=(MIN_SEGMENT_LENGTH, 120))
        bin_centers = (bins[:-1] + bins[1:]) / 2
        total = sum(counts)
        probs = counts / total if total > 0 else counts
        
        color = CITIES[city_key]['color']
        ax.bar(bin_centers, probs, width=(bins[1]-bins[0])*0.9, color=color, alpha=0.7, edgecolor='black', linewidth=0.5)
    
    ax.set_xlabel('Length (m)', fontsize=10)
    ax.set_ylabel('Probability' if idx == 0 else '', fontsize=10)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=11)
    ax.set_ylim(0, 0.2)
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Segment Length Distributions (Aligned)', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## A13. Node Degree Map (Spatial)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    
    if all_x and all_y:
        minx, maxx = min(all_x), max(all_x)
        miny, maxy = min(all_y), max(all_y)
        
        # Draw edges (light gray)
        for u, v in G.edges():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.4, zorder=1)
        
        # Draw nodes sized and colored by degree
        degrees = dict(G.degree())
        
        # Color map for degrees
        degree_colors = {1: 'red', 2: 'orange', 3: 'yellow', 4: 'green', 5: 'blue', 6: 'purple'}
        
        for node in G.nodes():
            degree = degrees[node]
            size = 30 + degree * 20
            color = degree_colors.get(degree, 'gray')
            
            ax.scatter(
                pos[node][0], pos[node][1],
                s=size, c=color, zorder=2,
                edgecolors='black', linewidths=0.5, alpha=0.8
            )
        
        ax.set_xlim(minx - 20, maxx + 20)
        ax.set_ylim(miny - 20, maxy + 20)
    
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nNode Degree Map", fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Node Degree Maps (size & color by degree)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A14. Core + Corridor Overlap Map

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

TOP_PERCENT = 0.2  # Top 20%

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    local_int = data['syntax']['local_integration']
    choice = data['syntax']['choice']
    
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    
    if all_x and all_y:
        minx, maxx = min(all_x), max(all_x)
        miny, maxy = min(all_y), max(all_y)
        
        # Find top X% for each metric
        int_values = sorted(local_int.values(), reverse=True)
        choice_values = sorted(choice.values(), reverse=True)
        
        int_threshold = int_values[int(len(int_values) * TOP_PERCENT)] if int_values else 0
        choice_threshold = choice_values[int(len(choice_values) * TOP_PERCENT)] if choice_values else 0
        
        # Classify nodes
        high_int = {n for n, v in local_int.items() if v >= int_threshold}
        high_choice = {n for n, v in choice.items() if v >= choice_threshold}
        overlap = high_int & high_choice
        
        # Draw edges (light gray)
        for u, v in G.edges():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.3, zorder=1)
        
        # Draw all nodes (small, gray)
        for node in G.nodes():
            ax.scatter(pos[node][0], pos[node][1], s=10, c='lightgray', zorder=2, alpha=0.5)
        
        # Highlight high integration (blue)
        for node in high_int - overlap:
            ax.scatter(pos[node][0], pos[node][1], s=60, c='blue', zorder=3, 
                      edgecolors='black', linewidths=0.5, alpha=0.7, label='High Integration')
        
        # Highlight high choice (green)
        for node in high_choice - overlap:
            ax.scatter(pos[node][0], pos[node][1], s=60, c='green', zorder=3,
                      edgecolors='black', linewidths=0.5, alpha=0.7, label='High Choice')
        
        # Highlight overlap (red)
        for node in overlap:
            ax.scatter(pos[node][0], pos[node][1], s=80, c='red', zorder=4,
                      edgecolors='black', linewidths=1, alpha=0.9, label='Both')
        
        ax.set_xlim(minx - 20, maxx + 20)
        ax.set_ylim(miny - 20, maxy + 20)
    
    ax.set_aspect('equal')
    ax.set_title(
        f"{CITIES[city_key]['name']}\nCore+Corridor Overlap ({len(overlap)} nodes)",
        fontweight='bold', fontsize=12
    )
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='blue', edgecolor='black', label='High Integration'),
    Patch(facecolor='green', edgecolor='black', label='High Choice'),
    Patch(facecolor='red', edgecolor='black', label='Overlap (Both)')
]
fig.legend(handles=legend_elements, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 0.99))

plt.suptitle('Core + Corridor Overlap (Top 20%)', fontsize=16, fontweight='bold', y=0.975)
plt.tight_layout()
plt.show()

## Save Reference Data

In [ ]:
with open('reference_cities_data.pkl', 'wb') as f:
    pickle.dump(city_data, f)

print("✓ Reference data saved to: reference_cities_data.pkl")
print("\nReady for Step 2: Network Generation")